In [3]:
from pathlib import Path
import requests

def download_one_file_of_raw_data(year: int, month: int) -> Path:
    URL = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet'
    response = requests.get(URL)

    if response.status_code == 200:
        path = f'../data/raw/rides_{year}-{month:02d}.parquet'
        open(path, "wb").write(response.content)
        return path
    else:
        raise Exception(f'URL {URL} is not available')

In [4]:
download_one_file_of_raw_data(year=2022, month=3)

'../data/raw/rides_2022-03.parquet'

In [5]:
import pandas as pd

rides=pd.read_parquet('../data/raw/rides_2022-02.parquet')
rides.head()


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.40,1.0,N,138,252,1,17.0,1.75,0.5,3.90,0.00,0.3,23.45,0.0,1.25
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.40,1.0,N,138,41,2,21.0,1.75,0.5,0.00,6.55,0.3,30.10,0.0,1.25
2,1,2022-02-01 00:03:20,2022-02-01 00:26:59,1.0,12.50,1.0,N,138,200,2,35.5,1.75,0.5,0.00,6.55,0.3,44.60,0.0,1.25
3,2,2022-02-01 00:08:00,2022-02-01 00:28:05,1.0,9.88,1.0,N,239,200,2,28.0,0.50,0.5,0.00,3.00,0.3,34.80,2.5,0.00
4,2,2022-02-01 00:06:48,2022-02-01 00:33:07,1.0,12.16,1.0,N,138,125,1,35.5,0.50,0.5,8.11,0.00,0.3,48.66,2.5,1.25


In [6]:
rides = rides[['tpep_pickup_datetime', 'PULocationID']]

rides.head()

,tpep_pickup_datetime,PULocationID
0,2022-02-01 00:06:58,138
1,2022-02-01 00:38:22,138
2,2022-02-01 00:03:20,138
3,2022-02-01 00:08:00,239
4,2022-02-01 00:06:48,138


In [7]:
rides.rename(columns={
    'tpep_pickup_datetime': 'pickup_datetime',
    'PULocationID': 'pickup_location_id',
}, inplace=True)

rides['pickup_datetime'].describe()

count                       2979431
mean     2022-02-15 07:55:36.811103
min             2003-01-01 00:10:06
25%             2022-02-08 15:30:42
50%             2022-02-15 11:52:28
75%             2022-02-22 12:31:14
max             2022-05-24 17:41:50
Name: pickup_datetime, dtype: object

In [8]:
rides['pickup_datetime'] = pd.to_datetime(rides['pickup_datetime'])

rides=rides[rides.pickup_datetime >='2022-02-01']
rides=rides[rides.pickup_datetime <='2022-02-28']

rides['pickup_datetime'].describe()

count                       2877147
mean     2022-02-14 20:58:45.475700
min             2022-02-01 00:00:00
25%             2022-02-08 11:11:58
50%             2022-02-14 20:10:07
75%      2022-02-21 15:08:44.500000
max             2022-02-28 00:00:00
Name: pickup_datetime, dtype: object

In [9]:
rides.to_parquet('../data/etl/val_rides_2022-02.parquet')

Agrupar x hora

In [10]:
import pandas as pd

rides=pd.read_parquet('../data/etl/val_rides_2022-02.parquet')


In [11]:
rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('h')
rides

,pickup_datetime,pickup_location_id,pickup_hour
0,2022-02-01 00:06:58,138,2022-02-01 00:00:00
1,2022-02-01 00:38:22,138,2022-02-01 00:00:00
2,2022-02-01 00:03:20,138,2022-02-01 00:00:00
3,2022-02-01 00:08:00,239,2022-02-01 00:00:00
4,2022-02-01 00:06:48,138,2022-02-01 00:00:00
...,...,...,...
2976465,2022-02-27 23:02:27,265,2022-02-27 23:00:00
2976466,2022-02-27 23:34:00,148,2022-02-27 23:00:00
2976467,2022-02-27 23:45:26,45,2022-02-27 23:00:00
2976468,2022-02-27 23:06:00,141,2022-02-27 23:00:00


In [12]:
agg_rides = rides.groupby(['pickup_hour', 'pickup_location_id']).size().reset_index()
agg_rides=agg_rides[agg_rides.pickup_location_id== 43]
agg_rides

,pickup_hour,pickup_location_id,0
10,2022-02-01 00:00:00,43,3
83,2022-02-01 01:00:00,43,1
144,2022-02-01 02:00:00,43,1
201,2022-02-01 03:00:00,43,3
306,2022-02-01 05:00:00,43,6
...,...,...,...
61718,2022-02-27 19:00:00,43,37
61822,2022-02-27 20:00:00,43,30
61912,2022-02-27 21:00:00,43,34
62010,2022-02-27 22:00:00,43,20


In [15]:
agg_rides.to_parquet('../data/transformed/ts_data_2022_02.parquet')


In [13]:
import plotly.express as px

# Rename count column (if needed)
agg_rides = agg_rides.rename(columns={0: 'ride_count'})

# Filter for location 43 and up to 15 Feb
filtered = agg_rides[
    (agg_rides['pickup_location_id'] == 43) &
    (agg_rides['pickup_hour'] < pd.to_datetime('2022-02-08'))
].copy()

# Add day of week (optional for color), but keep full datetime
filtered['day_of_week'] = filtered['pickup_hour'].dt.day_name()

# Plot with full timestamp on x-axis
fig = px.line(
    filtered,
    x='pickup_hour',  # full timestamp
    y='ride_count',
    color='day_of_week',  # for color-coded days
    title='Ride Counts Over Time (Location CP)',
    labels={
        'pickup_hour': 'Date & Hour',
        'ride_count': 'Number of Rides',
        'day_of_week': 'Day of Week'
    }
)
fig.update_layout(
    xaxis=dict(
        tickformat="%H",  # e.g. Feb 01, 00:00 (with line break for cleaner display)
        dtick=3600000,                # 1 hour in milliseconds
        tickangle=45,
        showgrid=True
    )
)

fig.show()

